<a href="https://colab.research.google.com/github/tluxxx/vbt_basics/blob/main/vbt_test_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction into vectorbt - part 3
* valid for latest free version (version: 0.28.1)
* portfolio with one ticker

Content:
* callback-functions in vectorbt
* customized stop-loss and take-profits, using callbacks


In [1]:
!pip install vectorbt ta python-dotenv --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.7/420.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.5 MB/s eta 0:00:00


In [2]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from numba import njit

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import yfinance as yf
import ta

## 0. Helpers functions

In [3]:
import sys
import importlib
import os
import helper

from google.colab import drive
from dotenv import load_dotenv

# connnecting Google-Drive, loading .env-file and getting path_data
from pathlib import Path
drive.mount('/content/drive')
load_dotenv('/content/drive/My Drive/.env')
path_data = os.getenv('PATH_VBT')             # replace with your path
sys.path.append(path_data)

# importing supporting functions (from helper-module in path_data)
for mod in ['timelines', 'entries_exits', 'plotting_support']:
    importlib.reload(importlib.import_module(f'helper.{mod}'))

from helper.timelines import get_timeline_basic, get_timeline_extended
from helper.entries_exits import lense_at_entry_exit, classify_exit_types
from helper.plotting_support import _is_boolean_series, plot_multi_subplot_trading_data

Mounted at /content/drive


## 1. DataDownload and Signal Generation

As in the previous parts we will:
* use direct price-data download from yfinance
* calculate indicators with the package ta
* implement a SMA-crossing LongOnly strategy for a portfolio with one ticker

In [4]:
# download data
ticker = 'AAPL'
start_date = '2022-01-01'
end_date = '2025-10-11'
prices = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# preparing for ta & vbt-use (remove MultiLevel-column-index, transform to a pd.Series)
close = prices.Close[ticker]
open = prices.Open[ticker]
high = prices.High[ticker]
low = prices.Low[ticker]

# entry and exit signals, generated by ta
fast_window = 10
slow_window = 60

fast_sma = ta.trend.sma_indicator(close, window=fast_window)
slow_sma = ta.trend.sma_indicator(close, window=slow_window)

# entry and exit Signals by simple SMA-crossing
entries = ((fast_sma > slow_sma) & (fast_sma.shift(1) < slow_sma.shift(1))).rename(ticker)
exits = ((fast_sma < slow_sma) & (fast_sma.shift(1) > slow_sma.shift(1))).rename(ticker)


[*********************100%***********************]  1 of 1 completed


## 2. working with callback-functions in vbt
Callback functions in vbt let you customize certain calculations during backtesting. Covering this advanced topic in full is beyond the scope of this article, so we'll focus on one practical use case: customizing exits with TakeProfit and StopLoss rules.

Callback functions run on every bar and calculate certain data, through the context object (`c`), which the callback function can access directly.

To preserve vbt's high-performance advantages, these functions must be fully numba-compatible.

### 2.1 context object (c)
The context-object (c) is an instance of `AdjustSLContext`. The context-object (c) is a imutable, named tuple, containing attributes for each step (=bar). We can cluster these attributes into:
* attributes for location,
* attributes for stop-price-calculation,
* price attributes and,
* trade attributes.

These attributes can be accessed as follows:

* attributes for locations
    * `c.i`: current row number within the c (array)
    * `c.col`: current row number within c (array)

* attributes for stop-price-calculations:
    * `c.curr_stop`: equals to `sl_loss`-value, used during generation of an portfolio-instance by `vbt.Portfolio.from_order(...)`. Shows the percentage of stop, if in position or equals to NaN if not in position.
    * `c.curr_trail`: equals to `sl_trail`-value, used during generation of an portfolio-instance. Is True if trailing and False otherwise.

* price attributes
    * `c.val_price_now`: current price, used for calculation/validation (default: close)

* trade attributes:
    * `c.init_i`: row number (in the array c) where current trade starts, -1 otherwise
    * `c.position_now`: equals to nb. of shares, if in position, NaN otherwise
    * `c.curr_price`: trailing-price in the current trades. It is calculated as curr_price [i] = maximum(high[i-1],curr_price [i-1]), once a full set of OHLC-data are provided. If only close prices are given, the calculation is curr_price [i] = maximum(close [i-1], curr_price [i-1]).
    
Note 1: `curr_price`is a **trailing parameter only**. To calculate the true-value of the current stop loss, we have to caclulate: true-value-SL = cur_price [i] * ( 1- `sl_stop`)

NOTE 2: in the contect-object, there are no specific built in attributes for TakeProfit stops. The reason is simple: The true value of the TakeProfit limit, can be calculated by: true-value-TP = cur_price[i] * (1 + `tp_stop`). Also note, that TakeProfit-limits do not have a built in option for trailing.

### 2.2 logging the content of the context-object (c)
As a first step, we need to understand better, what data is stored in this context-object.  

We write for the parameter `adjust_sl_func_nb` in the `.from_signals(...)`-class-method a callback-function `adjust_sl_func_nb_01`. The only purpose of this function is to log the content of the context object for our inspection.    

Important Note: The callback functions requires adhearance to strict formats and rules, mainly to make sure, they can processed by numba. On of the rules is, that this function has to (and can only) return two values: `c.curr_stop` and `c.curr_trail`. If we want to log the context-object and later process it outside the callback-function, we cannot simply return it via the function.

Instead we have to initialize a global logger-object and pass it as argument to the callback-function. The global object is modified (in our case filled with data) inside the function. Since it is global, it can be process later also outside of the function.

Another rule is always to include the object c into the parameter list.

Having this in mind, the callback function for logging purposes looks like:

In [5]:
@njit
def adjust_sl_func_nb_01(c, logger: np.ndarray) -> tuple[float, float]:
    ''' returns available paramters of context object (c)
    Args:
        c: vectorbt context object
        logger (np.ndarray): numpy array to log the context object attributes
    Returns:
        tuple[float, float]: current stop loss and current trail
    '''
    i = c.i
    logger[i, 0] = c.val_price_now
    logger[i, 1] = c.init_price
    logger[i, 2] = c.curr_stop
    logger[i, 3] = c.curr_trail
    logger[i, 4] = c.i
    logger[i, 5] = c.init_i
    logger[i, 6] = c.col
    logger[i, 7] = c.position_now
    logger[i, 8] = c.curr_price

    return c.curr_stop, c.curr_trail

In the following code block we intialize the logger first.  

Then we will generate a portfolio instance with a trailing StopLoss (here, value set to 5%) and with no TakeProfit-limit. Then we log the content of the context-object (c).

This requires to pass to:
* `adjust_sl_func_nb` the function `adjust_sl_func_nb_01`
* `adust_sl_args` the parameters, required for calling `adjust_func_sl_nb_01` (except the c !!!)

Note: `adjust_sl_args` requires tuples, therefore, we have to handover in our case (logger_sl,) and not (logger_sl)   

In [6]:
# initializing logger
shape = close.shape
logger_sl = np.zeros((shape[0], 9))

# Paramters for TakeProfit and StopLoss
stop_loss = 0.05            # % 5%
stop_loss_trail = True      # trailing-stop-loss
take_profit = np.inf        # no take profit

# generate  portfolio instance for logging context object
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    sl_stop=stop_loss,
    sl_trail=stop_loss_trail,
    adjust_sl_func_nb=adjust_sl_func_nb_01,
    adjust_sl_args=(logger_sl,),
    tp_stop=take_profit,
    high=high,
    low=low,
    freq='D',
    init_cash=10_000,
)

We now extract the trades, including the classification of exits (see part 2 for details).

Next, we get the timeline of parameters of interest (see also part 2). We will add to the standard content of the timeline (OHLC-data, entry/exit signals, effective entries/exits) the content of the logger.

Finaly we will add the current value of the StopLoss-price (that will in reality trigger the stop) that needs to be calculated from `sl_stop`and `c.curr_price`.

In [7]:
# list of trades with key data, amended by the trade type
trades = classify_exit_types(
    trades=pf.trades.records_readable,
    take_profit=take_profit,
    exits=exits
)

# generating timeline for standard + specific trading parameters
timeline = get_timeline_extended(prices, entries, exits, trades)    # standards (prices, entry/exit-signals, effective entries and exits)
df_log_sl = pd.DataFrame(logger_sl, index=close.index,              # specifics: adding the logger
                         columns=['val_price_now', 'init_price', 'cur_stop', 'cur_trail', 'i', 'init_i', 'col', 'position_now', 'curr_price'])
timeline = pd.concat([timeline, df_log_sl], axis=1)
timeline['sl_calc'] = timeline['curr_price'] * (1 - stop_loss)      # adding calculated sl

In [8]:
trades.head(3)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id,exit_type
0,0,AAPL,67.429307,2022-07-19,148.303467,0.0,2022-08-22,164.582420,0.0,1097.678514,0.109768,Long,Closed,0,stop_loss
1,1,AAPL,74.568745,2022-11-23,148.824799,0.0,2022-11-28,141.383559,0.0,-554.883926,-0.050000,Long,Closed,1,stop_loss
2,2,AAPL,74.838084,2023-01-30,140.874725,0.0,2023-02-10,147.288934,0.0,480.027052,0.045531,Long,Closed,2,stop_loss


Our timeline-dataframe will contain the following columns:

In [9]:
timeline.columns

Index(['open', 'high', 'low', 'close', 'sys_entries', 'sys_exits',
       'eff_entries', 'eff_exits_system', 'eff_exits_stop_loss',
       'eff_exits_take_profit', 'val_price_now', 'init_price', 'cur_stop',
       'cur_trail', 'i', 'init_i', 'col', 'position_now', 'curr_price',
       'sl_calc'],
      dtype='object')

Note: we again refrain from extensive usage of screenshots and limit ourself to a description of the behaviour of `Portfolio`, also using the logged values in the timeline. For further details and varification see the acompanying notebook in the GitHub.

### 2.3 Details about triggering StopLoss  

In Part 2 we were not able to demonstrate the details of triggering a StopLoss from the presented data. Now we have more details available (i.e. data from the logger objects) and we can inspect the behaviour of our portfolio instance in detail.

Inspecting the timeline around the first **Entry-Signal**, we observe:
* entry signal at bar [i]
* init_price [i] = NaN, init_price [i+1] = close [i]
* cur_stop [i] = NaN, cur_stop [i+1] = 0.05 (=`sl_stop`)
* cur_trail [i] = 0.0, cur_trail [i+1] = 1.0 (corresponds to `sl_trail`= True)
* init_i [i] = -1.0, init_i [i+1] = i
* position_now[i] = 0.0, position_now [i+1] = 67.43 (= # of assets)
* cur_price [i] = NaN, cur_price [i+1] = close [i]
* sl_calc [i] = NaN, sl_calc [i+1] = cur_price * (1 - `sl_stop`) (Remember: this parameter is calculated and was not logged from c!)

If we inspect timeline at **in position bars**, e.g bar [k], we find:
* init_price [k] remains unchanged
* cur_stop [k] and cur_trail [k] remains unchanged (at values of `sl_stop` and `sl_trail`)
* init_i [k] remains at i
* position_now [k] remains unchanged
* cur_price [k] = max(high [k-1], cur_price [k-1])
* sl_calc [k] = cur_price [k] * ( 1 - `stop_sl`)

Important: Except of the first bar in a trade, the trailing-parameter (cur_price) at a bar [k] is cacluated using **ONLY** the **maximum** of the previous **highs** and **cur_pos** until the **previous (!!) bar** in the ** current position**.

If this cur_price [k] is multiplied by (1 + `tp_stop`) we get the true value of the TakeProfit, if multiplied by (1 - `sl_stop`) we are getting the true value of the (trailing) StopLoss. As a conseqence these stops/limits are a bit more conservative (higher TakeProfit-limits, higher StopLoss-prices), compared to stops, calculated based on **close**.

Now lets check, how an **Exit Signal** is triggered in case of a trailing stop loss (at bar [j]):
* all parameters are updated, according to the rules, described for **in position bars**  
* bar [j] is the first bar, where low [j] < cur_price [j] * (1 - `sl_stop`). This triggers a StopLoss Signal, base on the lows of the asset prices (!!).

After the Exit-Signal at bar [j] we can observe at bar [j+1]:
* init_price [j+1] = NaN
* cur_stop [j+1] = NaN, cur_trail [j+1] = 0.0
* init_i [j+1] = -1.0
* position_now [j+1] = 0.0
* cur_pos [j+1] = NaN
* sl_calc [j+1] = NaN

Next bar after an effective exit-signal, the parameters will be in a state, ready for the next effective entry signal.    

In [10]:
# timeline around entry and exit of the first trade
entry_date = '2022-07-19'
exit_date = '2022-08-22'
lense_at_entry_exit(timeline, entry_date, exit_date, 1)

### Timeline around Entry Date: 2022-07-19 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,val_price_now,init_price,cur_stop,cur_trail,i,init_i,col,position_now,curr_price,sl_calc
Date,,,,,,,,,,,,,,,,,,,,
2022-07-18,148.048140,148.863320,144.080277,144.443680,False,False,False,False,False,False,144.443680,NaN,NaN,0.0,134.0,-1.0,0.0,0.000000,NaN,NaN
2022-07-19,145.278467,148.529355,144.286509,148.303467,True,False,True,False,False,False,148.303467,NaN,NaN,0.0,135.0,-1.0,0.0,0.000000,NaN,NaN
2022-07-20,148.421341,150.974917,147.684734,150.307053,False,False,False,False,False,False,150.307053,148.303467,0.05,1.0,136.0,135.0,0.0,67.429307,148.303467,140.888293




### Timeline around Exit Date: 2022-08-22 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,val_price_now,init_price,cur_stop,cur_trail,i,init_i,col,position_now,curr_price,sl_calc
Date,,,,,,,,,,,,,,,,,,,,
2022-08-19,170.176129,170.874426,168.484497,168.691040,False,False,False,False,False,False,168.691040,148.303467,0.05,1.0,158.0,135.0,0.0,67.429307,173.244652,164.58242
2022-08-22,166.891227,167.058421,164.383283,164.806198,False,False,False,False,True,False,164.806198,148.303467,0.05,1.0,159.0,135.0,0.0,67.429307,173.244652,164.58242
2022-08-23,164.324282,165.927402,163.901366,164.471802,False,False,False,False,False,False,164.471802,NaN,NaN,0.0,160.0,-1.0,0.0,0.000000,NaN,NaN


## 3. extracting take-profit and stop-loss-values using callbacks

Besides the parameter `adjust_sl_func_nb`, that allows to specify a callback-function for StopLoss-caclulation, we also have a parameter `adjust_tp_func` that allows to specify a callback function for TakeProfits.

In our next example we will use these function to return:
* the value of a (trailing) StopLoss
* the entry value of a position
* the value of a TakeProfit-limit

This will allow us to plot all of them in a chart, for a better visualisation.

First we write the two required callback-functions:  

In [11]:
@njit
def adjust_sl_func_nb_02(c, sl_price: np.ndarray, init_price: np.array) -> tuple[float, float]:
    ''' Records current SL stop and initial_price
    Args:
        c: AdjustSLContext (context for current row/column)
        sl_price: array for logging the Stop-Loss-prices
    Returns:
        tuple[float, float]: current stop loss and current trail
    '''
    i = c.i
    sl_price[i] = c.curr_price * (1 - stop_loss)
    init_price[i] = c.init_price

    return c.curr_stop, c.curr_trail

In [12]:
@njit
def adjust_tp_func_nb_02(c, tp_stop, tp_price: np.ndarray) -> float:
    ''' Records current TP stop,
    Args:
        c: AdjustTPContext (context for current row/column)
        tp_stop: current value for taking profit limits (%)
        tp_price: array for logging Take-Profit-Prices
    Returns:
        float: the current TP stop (scalar)
    '''
    i = c.i
    if c.init_i == -1:
        tp_price[i] = np.nan
    else:
        tp_price[i] = c.init_price * (1 + tp_stop)

    return c.curr_stop

We initialize our loggers (for the parameters of interest), define the TakeProfit and StopLoss-parameters and generate an instance of the portfolio-class as follows:

In [13]:
# initializing logger
shape = close.shape
sl_price = np.zeros(shape[0])
in_price = np.zeros(shape[0])
tp_price = np.zeros(shape[0])

# Paramters for TakeProfit and StopLoss
stop_loss = 0.05            # 5%
stop_loss_trail = True      # trailing-stop-loss
take_profit = 0.15          # 15%

# generate a portfolio instance with trailing stop loss and take profit
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    sl_stop=stop_loss,
    sl_trail=stop_loss_trail,
    adjust_sl_func_nb=adjust_sl_func_nb_02,
    adjust_sl_args=(sl_price, in_price),
    tp_stop=take_profit,
    adjust_tp_func_nb=adjust_tp_func_nb_02,
    adjust_tp_args=(take_profit, tp_price),
    high=high,
    low=low,
    freq='D',
    init_cash=10_000,
)

We again tabulate the trades (including exit classification) and generate a specific timeline with the data of interest:

In [14]:
# list of trades with key data, amended by the trade type
trades = classify_exit_types(
    trades=pf.trades.records_readable,
    take_profit=take_profit,
    exits=exits
)

# generating timeline for standard + specific trading parameters
timeline = get_timeline_extended(prices, entries, exits, trades)    # standards (prices, entry/exit-signals, effective entries and exits)

timeline['fast_sma'] = fast_sma
timeline['slow_sma'] = slow_sma
timeline['sl_price'] = sl_price  # adding our logged parameters
timeline['in_price'] = in_price
timeline['tp_price'] = tp_price

timeline['position'] = np.sign(pf.net_exposure())

In [15]:
trades

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id,exit_type
0,0,AAPL,67.429307,2022-07-19,148.303467,0.0,2022-08-16,170.548987,0.0,1500.000000,0.150000,Long,Closed,0,take_profit
1,1,AAPL,77.272068,2022-11-23,148.824799,0.0,2022-11-28,141.383559,0.0,-575.000000,-0.050000,Long,Closed,1,stop_loss
2,2,AAPL,77.551172,2023-01-30,140.874725,0.0,2023-02-10,147.288934,0.0,497.429358,0.045531,Long,Closed,2,stop_loss
3,3,AAPL,61.857846,2023-11-10,184.656113,0.0,2024-01-02,183.903229,0.0,-46.571777,-0.004077,Long,Closed,3,stop_loss
4,4,AAPL,58.834366,2024-01-23,193.353958,0.0,2024-01-31,182.674835,0.0,-628.299429,-0.055231,Long,Closed,4,stop_loss
5,5,AAPL,59.293159,2024-05-08,181.261353,0.0,2024-06-12,211.632629,0.0,1800.808940,0.167555,Long,Closed,5,take_profit
6,6,AAPL,55.499796,2024-09-24,226.097534,0.0,2024-10-07,220.111214,0.0,-332.239568,-0.026477,Long,Closed,6,stop_loss
7,7,AAPL,53.586805,2024-11-20,227.968948,0.0,2025-01-02,242.752106,0.0,792.182166,0.064847,Long,Closed,7,stop_loss
8,8,AAPL,52.823924,2025-02-24,246.257919,0.0,2025-02-27,236.491318,0.0,-515.910222,-0.039660,Long,Closed,8,stop_loss
9,9,AAPL,58.928173,2025-07-02,211.993668,0.0,2025-08-01,201.954819,0.0,-591.571021,-0.047354,Long,Closed,9,stop_loss


In [17]:
# plot configuration 01:
# 1. Subplot: Candlesticks, SMA's, StopLossPrice, TakeProfitPrice, InitialPrice, effective Entry/Exit-Signals;
# 2. Subplot: Positions

subplot_config_02 = [
    {"title": "Prices, Indicators, effective Signals", "height_ratio": 8,
     "traces": [
          {"type": "candlestick", "open": "open", "high": "high", "low": "low", "close": "close"},
          {"type": "scatter", "y": "fast_sma", "name": "SMA Fast", "line": {"color": "orange"}},
          {"type": "scatter", "y": "slow_sma", "name": "SMA Slow", "line": {"color": "blue"}},
          {"type": "scatter", "y": "sl_price", "name": "Stop Loss Price", "line": {"color": "red"}},
          {"type": "scatter", "y": "tp_price", "name": "Take Profit Price", "line": {"color": "grey"}},
          {"type": "scatter", "y": "in_price", "name": "Initial Price", "line": {"color": "green"}},
          {"type": "scatter_markers", "y": "eff_entries", "marker": {"color": "green", "symbol": "triangle-up", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_system", "marker": {"color": "red", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_stop_loss", "marker": {"color": "blue", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_take_profit", "marker": {"color": "grey", "symbol": "triangle-down", "size": 15}},
        ],
    },
    { "title": "Position", "height_ratio": 1,
     "traces": [
         {"type": "scatter", "y": "position", "name": "Position", "line": {"color": "green", "width": 2},
          "line_shape": "hv",   # <-- stepped line
          "mode": "lines"       # ensure 'lines' mode is used
          }
         ],
    },
]

Again we use our flexible plotting function `plot_multi_subplot_trading_data` to generate an overview chart, containing:
* Candlestick-Chart with fast-SMA and Slow-SMA
* all entries
* all exits (per type)
* all Stop-Loss-Level and Take-Profit-Limits (if in position)
* relevant entry-price,  if in position
* the current trading position (in a separate subplot)

In [18]:
fig = plot_multi_subplot_trading_data(
    timeline,
    subplot_config_02,
    title = f'Overview for {ticker}, from {start_date} to {end_date}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()

Next, we will magnify a certain section of the timeline, to get a better visualisation:

In [19]:
# focusing on an intervall for analyses
start_analyses='2024-05-01'
end_analyses='2024-06-15'
timeline_slice = timeline.loc[start_analyses:end_analyses]

In [20]:
timeline_slice

,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,tp_price,position
Date,,,,,,,,,,,,,,,,
2024-05-01,168.207848,171.312526,167.741650,167.930115,False,False,False,False,False,False,167.247676,173.400938,NaN,NaN,NaN,0.0
2024-05-02,171.114101,172.016742,169.507215,171.629898,False,False,False,False,False,False,167.841826,173.135953,NaN,NaN,NaN,0.0
2024-05-03,185.139693,185.486867,181.181989,181.896164,False,False,False,False,False,False,169.664952,173.040257,NaN,NaN,NaN,0.0
2024-05-06,180.874536,182.709558,178.960145,180.239716,False,False,False,False,False,False,171.239114,172.934949,NaN,NaN,NaN,0.0
2024-05-07,181.965594,183.403859,179.852840,180.924088,False,False,False,False,False,False,172.776572,172.828319,NaN,NaN,NaN,0.0
2024-05-08,181.370463,181.588684,179.981782,181.261353,True,False,True,False,False,False,174.137468,172.755413,NaN,NaN,NaN,1.0
2024-05-09,181.082808,183.165821,180.636452,183.076553,False,False,False,False,False,False,175.593590,172.747644,172.198285,181.261353,208.450555,1.0
2024-05-10,183.652620,183.841340,180.901318,181.815109,False,False,False,False,False,False,176.982089,172.733563,174.007530,181.261353,208.450555,1.0
2024-05-13,184.188986,185.837791,183.374510,185.023315,False,False,False,False,False,False,178.274809,172.777747,174.649273,181.261353,208.450555,1.0


In [21]:
fig = plot_multi_subplot_trading_data(
    timeline_slice,
    subplot_config_02,
    title = f'slice for {ticker}, from {start_analyses} to {end_analyses}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()

## 4. Customized stops
### 4.1. Stop-loss depending from trade-profit

Now lets use our experience, to customize stops.

We assume a StopLoss logic as follows:

* at every bar we caclulate the current trade-profit (i.e current_price - inital_price)
* if trade-profit <= 10%, we set `sl_stop` = 10% and `sl_trail` = False
* if (10% < trade-profit <=20%), we tighten the stop to `sl_stop` = 5% and keep `sl_trail` = False
* if our trade gets very profitable (i.e. trade-profit > 20%) we keep `sl_stop` = 5%, but set `sl_trail` = True.

This is a system that thightens the StopLoss, the more profitable the trade gets.

To achieve this, we will use the following callback-function:

In [22]:
@njit
def adjust_sl_func_nb_03(c, sl_price: np.ndarray, init_price: np.ndarray, trade_profit: np.ndarray) -> tuple[float, float]:
    ''' Adjust stop-loss price and trailing behavior dynamically based on current trade profit
        Note: this version only covers LongOnly-Strategies
    Args:
        c:            AdjustSLContext (context for current row/column)
        sl_price:     array to record the stop-loss price
        init_price:   array to record the initial entry price
        trade_profit: array to record the trade-profit
    Returns:
        tuple[float, float]: (current stop loss fraction, trailing flag)
    '''
    # Current trade profit (fraction, e.g., 0.12 = +12%)

    i = c.i

    ctp = np.nan
    if not np.isnan(c.init_price):          # in active trade
        ctp = (c.val_price_now - c.init_price) / c.init_price

    # Determine SL and trailing behavior, depending o current trailing behaviour
    if ctp < 0.10:
        sl_stop = 0.10
        sl_trail = False
    elif ctp <= 0.20:
        sl_stop = 0.05
        sl_trail = False
    else:  # ctp > 0.20
        sl_stop = 0.05
        sl_trail = True

    # Record SL and initial prices for later analysis/plotting
    sl_price[i] = max(c.curr_price * (1 - sl_stop), sl_price[i-1])
    init_price[i] = c.init_price
    trade_profit[i] = ctp

    return sl_stop, sl_trail

Then we initialize the loggers, set the parameters and generate the instance of `Portfolio`:

In [23]:
# initializing logger
shape = close.shape
sl_price = np.zeros(shape[0])
in_price = np.zeros(shape[0])
tp_price = np.zeros(shape[0])
trade_profit = np.zeros(shape[0])

# No Paramters for Stop Loss (controlled by callback), no take_profit
take_profit = np.inf

# generate a portfolio instance with customized (trailing) StopLoss
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    adjust_sl_func_nb=adjust_sl_func_nb_03,
    adjust_sl_args=(sl_price, in_price, trade_profit),
    high=high,
    low=low,
    freq='D',
    init_cash=10_000,
)

In [24]:
# list of trades with key data, amended by the trade type
trades = classify_exit_types(
    trades=pf.trades.records_readable,
    take_profit=take_profit,
    exits=exits
)

# generating timeline for standard + specific trading parameters
timeline = get_timeline_extended(prices, entries, exits, trades)    # standards (prices, entry/exit-signals, effective entries and exits)

timeline['fast_sma'] = fast_sma
timeline['slow_sma'] = slow_sma
timeline['sl_price'] = sl_price
timeline['in_price'] = in_price
timeline['trade_profit'] = trade_profit

timeline['position'] = np.sign(pf.net_exposure())

In [25]:
# plot configuration 03:
# 1. Subplot: Candlesticks, SMA's, StopLossPrice, InitialPrice, effective Entry/Exit-Signals;
# 2. Subplot: current trade-profits
# 2. Subplot: Positions

subplot_config_03 = [
    {"title": "Prices, Indicators, effective Signals", "height_ratio": 8,
     "traces": [
          {"type": "candlestick", "open": "open", "high": "high", "low": "low", "close": "close"},
          {"type": "scatter", "y": "fast_sma", "name": "SMA Fast", "line": {"color": "orange"}},
          {"type": "scatter", "y": "slow_sma", "name": "SMA Slow", "line": {"color": "blue"}},
          {"type": "scatter", "y": "sl_price", "name": "Stop Loss Price", "line": {"color": "red"}},
          {"type": "scatter", "y": "in_price", "name": "Initial Price", "line": {"color": "green"}},
          {"type": "scatter_markers", "y": "eff_entries", "marker": {"color": "green", "symbol": "triangle-up", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_system", "marker": {"color": "red", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_stop_loss", "marker": {"color": "blue", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_take_profit", "marker": {"color": "grey", "symbol": "triangle-down", "size": 15}},
        ],
    },

    {"title": "Trade Profit", "height_ratio": 3,
     "traces": [
         {"type": "scatter", "y": "trade_profit", "name": "Trade-Profit", "line": {"color": "blueviolet", "width": 2},
          "mode": "lines",
          },
         ],
    },

    { "title": "Position", "height_ratio": 1,
     "traces": [
         {"type": "scatter", "y": "position", "name": "Position", "line": {"color": "green", "width": 2},
          "line_shape": "hv",   # <-- stepped line
          "mode": "lines"       # ensure 'lines' mode is used
          }
         ],
    },
]

In the plot we see the expected behaviour. When the trade profit (shown in the middle sub-plot) exceeds the trigger-values (10%, 20%), the stopp-loss behaviour is changed (first a tighter stop, then switch to trailing).

In [26]:
fig = plot_multi_subplot_trading_data(
    timeline,
    subplot_config_03,
    title = f'Overview for {ticker}, from {start_date} to {end_date}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()

In [28]:
# focusing on an interval for analyses
start_analyses='2022-07-01'
end_analyses='2023-08-20'
timeline_slice = timeline.loc[start_analyses:end_analyses]

We will focus on the third trade as example (the right trade on the plot)

The trade is entered at bar [i] (at '2023-01-30'). Since `sl_stop` = 0.1, the (initial) StopLoss value is set to 90% of the `init_price` (= close [i] * (1 - `sl_stop`). Since `sl_trail` is set to False, it remains there.

At bar [j] (at '2023-03_21'), the trade profit exceds the first threshold defined in the callback (trade-profit > 10%). In the callback the parameter `sl_stop` is now set to 5%, `sl_trail` remains False. As a consequence, the StopLoss-price is lifted up by 5 percent, as can be seen by the small step at bar [j].

At bar [k] (at '2023-05-05') the next threshold is reached (trade profit exceeds 20%). This triggers in the callback setting `sl_trail` to True (but  `sl_stop` remains at 5%). This changes the behaviour of the StopLoss completely. The large jump is obvious. By switching to trail-mode at bar [k], the StopLoss-price is completly readjusted, it is set to high [k] * (1 - `sl_stop`).

Finally at bar [m] (at '2023-08-04') the position is closed, since the low is below the trailing StopLoss price.

In [29]:
fig = plot_multi_subplot_trading_data(
    timeline_slice,
    subplot_config_03,
    title = f'slice for {ticker}, from {start_analyses} to {end_analyses}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()

In [30]:
# 'misuse of lense for inspecting timeline around entry and first increase of stop loss of the third trade
entry_date = '2023-01-30'   # = entry into position
exit_date = '2023-03-20'    # here first change of Stop-Loss, triggered by exceeding a threshold for trade-profit
lense_at_entry_exit(timeline, entry_date, exit_date, 1)

## note the terme entry and exit are misleading here, simply think of moment 1 and moment 2

### Timeline around Entry Date: 2023-01-30 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,trade_profit,position
Date,,,,,,,,,,,,,,,,
2023-01-27,141.032363,145.041867,140.953550,143.761185,False,False,False,False,False,False,137.371581,137.684364,NaN,NaN,NaN,0.0
2023-01-30,142.805602,143.386830,140.726961,140.874725,True,False,True,False,False,False,138.183336,137.562855,NaN,NaN,NaN,1.0
2023-01-31,140.579173,142.194798,140.165417,142.145538,False,False,False,False,False,False,139.005922,137.554648,126.787253,140.874725,0.009021,1.0




### Timeline around Exit Date: 2023-03-20 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,trade_profit,position
Date,,,,,,,,,,,,,,,,
2023-03-17,153.995110,154.646298,152.219151,152.929535,False,False,False,False,False,False,150.392874,141.264413,126.787253,140.874725,0.085571,1.0
2023-03-20,152.998592,155.711858,152.090868,155.297455,False,False,False,False,False,False,150.745102,141.680475,133.830989,140.874725,0.102380,1.0
2023-03-21,155.218549,157.270751,154.448954,157.152359,False,False,False,False,False,False,151.502843,142.075731,133.830989,140.874725,0.115547,1.0


In [31]:
# 'misuse of lense for inspecting timeline around first increase of stop loss and switch to trailing stop loss of the thrid trade
entry_date = '2023-03-20'   # first increase of the stop loss (10% --> 5% )
exit_date = '2023-05-08'    # switch to trailing behaviour
lense_at_entry_exit(timeline, entry_date, exit_date, 1)

## note the terme entry and exit are missleading, simply think of moment 1 and moment 2

### Timeline around Entry Date: 2023-03-20 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,trade_profit,position
Date,,,,,,,,,,,,,,,,
2023-03-17,153.995110,154.646298,152.219151,152.929535,False,False,False,False,False,False,150.392874,141.264413,126.787253,140.874725,0.085571,1.0
2023-03-20,152.998592,155.711858,152.090868,155.297455,False,False,False,False,False,False,150.745102,141.680475,133.830989,140.874725,0.102380,1.0
2023-03-21,155.218549,157.270751,154.448954,157.152359,False,False,False,False,False,False,151.502843,142.075731,133.830989,140.874725,0.115547,1.0




### Timeline around Exit Date: 2023-05-08 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,trade_profit,position
Date,,,,,,,,,,,,,,,,
2023-05-05,168.696051,171.971709,168.478988,171.251465,False,False,False,False,False,False,165.350368,156.286086,133.830989,140.874725,0.215629,1.0
2023-05-08,170.176024,171.527734,169.810972,171.182404,False,False,False,False,False,False,166.156453,156.661996,163.373124,140.874725,0.215139,1.0
2023-05-09,170.738441,171.221886,169.307813,169.475540,False,False,False,False,False,False,166.945769,157.003375,163.373124,140.874725,0.203023,1.0


### 4.2. Example 2: trailing StopLoss (inital setting, depending on ATR at entry)

The ATR is a measure for price volatility. The inital StopLoss price (at the entry of a position) is set to a lower value in case of higher volatilities (= high ATR) and vice verca. Therefore we will avoid in times of high volatilites an to early exit by StopLoss, however, in times of low volatility we will tighten the StopLoss.  

To reflect this, we will set the StopLoss at stop_loss[i] = close[i] x (1 - (factor * ATR[i])/close[i]), if we enter a position at bar [i].

In our strategy (factor *ATR) remains constant if in a trade.  

The callback-function looks as follows:

In [33]:
@njit
def adjust_sl_func_nb_04(c, sl_price: np.ndarray, init_price: np.ndarray, atr: np.ndarray, factor: float) -> tuple[float, bool]:
    ''' inital Stop-Loss set a init_price - factor * ATR, then trailing
        LongOnly Strategy
    Args:
        c:          AdjustSLContext
        sl_price:   array to record stop-loss price (for plotting)
        init_price: array to record entry price
        atr:        precomputed ATR array
        factor:     ATR multiplier (float)
    Returns:
        (sl_stop, sl_trail)
    '''
    i = c.i
    # New trade entry detected
    if not np.isnan(c.init_price) and (i == 0 or np.isnan(init_price[i - 1])):
        # compute fractional stop distance so absolute stop = init_price - factor*ATR
        sl_stop = (factor * atr[i-1]) / c.init_price   # use bar, where signal was generated
        sl_trail = True
        sl_price[i] = c.init_price - factor * atr[i-1]

    # Ongoing trade: do NOT recompute sl_stop from ATR — keep current
    elif not np.isnan(c.init_price):
        sl_stop = c.curr_stop
        sl_trail = True
        sl_price[i] = max(c.curr_price * (1 - sl_stop), sl_price[i-1])

    # No active trade
    else:
        sl_stop = np.nan
        sl_trail = False
        sl_price[i] = np.nan

    # record entry price (NaN when no trade)
    init_price[i] = c.init_price

    return sl_stop, sl_trail

After initializing the loggers, setting the StopLoss and TakeProfit parameters, defining the factor and calculating the ATR (using the function, provided by ta), we generate an instance of the `portfolio` class as follows:

In [34]:
# initializing logger
shape = close.shape
sl_price = np.zeros(shape[0])
in_price = np.zeros(shape[0])

# No Paramters for Stop Loss (controlled by callback), no take_profit
take_profit = np.inf

# ATR-calculation
factor = 3.0
window = 14
atr = ta.volatility.AverageTrueRange(high=high,low=low,close=close,
                                     window=window,
                                     ).average_true_range().to_numpy()

# generate a portfolio instance with customized (trailing)-stop
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    adjust_sl_func_nb=adjust_sl_func_nb_04,
    adjust_sl_args=(sl_price, in_price, atr, factor),
    high=high,
    low=low,
    init_cash=10_000,
    freq='D',
)

In [35]:
# list of trades with key data, amended by the trade type
trades = classify_exit_types(
    trades=pf.trades.records_readable,
    take_profit=take_profit,
    exits=exits
)
# generating timeline for standard + specific trading parameters
timeline = get_timeline_extended(prices, entries, exits, trades)

timeline['fast_sma'] = fast_sma
timeline['slow_sma'] = slow_sma
timeline['sl_price'] = sl_price
timeline['in_price'] = in_price
timeline['atr'] = atr

timeline['position'] = np.sign(pf.net_exposure())

In [36]:
# plot configuration 04:
# 1. Subplot: Candlesticks, SMA's, StopLossPrice, InitialPrice, effective Entry/Exit-Signals;
# 2. Subplot: ATR
# 3. Subplot: Positions

subplot_config_04 = [
    {"title": "Prices, Indicators, effective Signals", "height_ratio": 8,
     "traces": [
          {"type": "candlestick", "open": "open", "high": "high", "low": "low", "close": "close"},
          {"type": "scatter", "y": "fast_sma", "name": "SMA Fast", "line": {"color": "orange"}},
          {"type": "scatter", "y": "slow_sma", "name": "SMA Slow", "line": {"color": "blue"}},
          {"type": "scatter", "y": "sl_price", "name": "Stop Loss Price", "line": {"color": "red"}},
          {"type": "scatter", "y": "in_price", "name": "Initial Price", "line": {"color": "green"}},
          {"type": "scatter_markers", "y": "eff_entries", "marker": {"color": "green", "symbol": "triangle-up", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_system", "marker": {"color": "red", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_stop_loss", "marker": {"color": "blue", "symbol": "triangle-down", "size": 15}},
          {"type": "scatter_markers", "y": "eff_exits_take_profit", "marker": {"color": "grey", "symbol": "triangle-down", "size": 15}},
        ],
    },

    {"title": "ATR", "height_ratio": 3,
     "traces": [
         {"type": "scatter", "y": "atr", "name": "ATR", "line": {"color": "blueviolet", "width": 2},
          "mode": "lines",
          },
         ],
    },

    { "title": "Position", "height_ratio": 1,
     "traces": [
         {"type": "scatter", "y": "position", "name": "Position", "line": {"color": "green", "width": 2},
          "line_shape": "hv",   # <-- stepped line
          "mode": "lines"       # ensure 'lines' mode is used
          }
         ],
    },
]

In [37]:
fig = plot_multi_subplot_trading_data(
    timeline,
    subplot_config_04,
    title = f'Overview for {ticker}, from {start_date} to {end_date}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()

In [38]:
# focusing on an intervall of interest for analyses
start_analyses='2024-09-01'
end_analyses='2025-01-30'
timeline_slice = timeline.loc[start_analyses:end_analyses]

At the subplot in the middle we see the ATR-timeline.

It is evident, that the ATR at the entry of the first trade is larger than the ATR at the entry of the second trade. As result, the distance between intial-stop level and entry price is larger for the first trade as for the second trade. Both trades are stopped out by a trailing StopLoss.

In [39]:
fig = plot_multi_subplot_trading_data(
    timeline_slice,
    subplot_config_04,
    title = f'slice for {ticker}, from {start_analyses} to {end_analyses}',
    size=(1400, 500),
    theme='plotly',
)
fig.show()

In [40]:
# lense for inspecting timeline around entry and exit of the seventh trade
entry_date = '2024-09-24'
exit_date = '2024-11-01'
lense_at_entry_exit(timeline, entry_date, exit_date, 1)

### Timeline around Entry Date: 2024-09-24 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,atr,position
Date,,,,,,,,,,,,,,,,
2024-09-23,226.067717,228.165910,224.546281,225.202591,False,False,False,False,False,False,221.292584,221.308765,NaN,NaN,5.042976,0.0
2024-09-24,227.370370,228.066464,224.466713,226.097534,True,False,True,False,False,False,222.014520,221.590405,NaN,NaN,4.939888,1.0
2024-09-25,223.671191,226.017984,222.766295,225.103134,False,False,False,False,False,False,222.383441,221.753994,211.277869,226.097534,4.824985,1.0




### Timeline around Exit Date: 2024-11-01 (1 rows before and after)



,open,high,low,close,sys_entries,sys_exits,eff_entries,eff_exits_system,eff_exits_stop_loss,eff_exits_take_profit,fast_sma,slow_sma,sl_price,in_price,atr,position
Date,,,,,,,,,,,,,,,,
2024-10-31,228.056519,228.543782,224.108735,224.645721,False,False,False,False,False,False,231.015858,225.024468,220.68163,226.097534,4.088813,1.0
2024-11-01,219.733361,224.088853,219.037281,221.662506,False,False,False,False,True,False,229.813626,225.187660,220.68163,226.097534,4.197358,0.0
2024-11-04,219.753252,221.543166,218.480416,220.767532,False,False,False,False,False,False,228.374727,225.287432,NaN,NaN,4.124839,0.0
